## Advent of code 2022 day 21-25
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import functools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import sortedcontainers
#import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines, nostrip=False):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end,
    by default also strips all lines (if nostrip is set only strips empty lines)'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line_str=line.strip()
        if nostrip==False or len(line_str)<1:
            line=line_str
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2022 day 21
# mv ~/Downloads/input* data_src/2022-day-21-input.txt
# big input file looks like: 2597 rules
# idea: part 1 parse as tuples, then DFS
# part 2: take the two root parts, one varies with humn, the other doesn't,
# the variable part can be increasing or decreasing with humn, binary search to find humn

sample2='''
root: pppw + sjmn
dbpl: 5
cczh: sllz + lgvd
zczc: 2
ptdq: humn - dvpt
dvpt: 3
lfqf: 4
humn: 5
ljgn: 2
sjmn: drzm * dbpl
sllz: 4
pppw: cczh / lfqf
lgvd: ljgn * ptdq
drzm: hmdt - zczc
hmdt: 32
'''

def calc(nodename, rules):
    tup=rules[nodename]
    if len(tup)==1:
        return tup[0]
    assert len(tup)==3
    arg1=calc(tup[0], rules)
    arg2=calc(tup[2], rules)
    if tup[1]=='+':
        return arg1+arg2
    elif tup[1]=='-':
        return arg1-arg2
    elif tup[1]=='*':
        return arg1*arg2
    elif tup[1]=='/':
        return arg1/arg2
    else:
        assert False

sample1=open('data_src/2022-day-21-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ s.split() for s in lines ]
for tup in data:
    if tup[0].endswith(':'):
        tup[0]=tup[0][:-1]
    if len(tup)==2:
        tup[1]=int(tup[1])
data={ tup[0]: tup[1:] for tup in data }
# part 1
res=int(calc('root', data))
print(f'part 1: {res}')
print()

# part 2
# find parts of root
root1=data['root'][0]
root2=data['root'][2]
# try with different humn to see which is variable
data['humn']=[0]
root1a=calc(root1, data)
root2a=calc(root2, data)
data['humn']=[1]
root1b=calc(root1, data)
root2b=calc(root2, data)
assert (root1a==root1b) != (root2a==root2b) # could be tricky if both were variable..
if root1a==root1b:
    fixedroot=root1
    varroot=root2
    increasing= root2b>root2a
else:
    fixedroot=root2
    varroot=root1
    increasing= root1b>root1a
targetval=calc(fixedroot, data)
print(f'part 2: fixedroot={targetval}, {increasing=}')
# binary search with ever-increasing range
found=False
start0= -1000
while not found:
    start=start0
    end= -start
    while start<end-1:
        mid=(start+end)//2
        data['humn']=[mid]
        val=calc(varroot, data)
        if increasing:
            if val<targetval:
                start=mid
            else:
                end=mid
        else:
            if val<targetval:
                end=mid
            else:
                start=mid
    found=False
    for i in [start, end]:
        data['humn']=[i]
        val=calc(varroot, data)
        if int(val)==int(targetval):
            print(f'part 2: for humn={i} varroot={int(val)}')
            found=True
    if not found:
        start0*=1000

# part 1: 223971851179174
# part 2: 3379022190351

In [ ]:
# part 2 alternative solution: recursively reversing the tree

def check_fixed(tree, hnode, rules):
    '''does this tree have a fixed value (returns val) or depend on hnode value (returns None) ?'''
    rules[hnode]=[0]
    val=calc(tree, rules)
    rules[hnode]=[1]
    if calc(tree, rules)==val:
        return val
    else:
        return None

def solve(tree, targetval, hnode, rules):
    if tree==hnode:
        return targetval
    assert len(rules[tree])==3
    t1=rules[tree][0]
    oper=rules[tree][1]
    t2=rules[tree][2]
    if (v1:=check_fixed(t1, hnode, rules))!=None:
        if oper=='+':
            return solve(t2, targetval-v1, hnode, rules) # v1+?=targetval
        elif oper=='-':
            return solve(t2, v1-targetval, hnode, rules) # v1-?=targetval
        elif oper=='*':
            return solve(t2, targetval/v1, hnode, rules) # v1*?=targetval
        elif oper=='/':
            return solve(t2, v1/targetval, hnode, rules) # v1/?=targetval
    else:
        assert (v2:=check_fixed(t2, hnode, rules))!=None
        if oper=='+':
            return solve(t1, targetval-v2, hnode, rules) # ?+v2=targetval
        elif oper=='-':
            return solve(t1, targetval+v2, hnode, rules) # ?-v2=targetval
        elif oper=='*':
            return solve(t1, targetval/v2, hnode, rules) # ?*v2=targetval
        elif oper=='/':
            return solve(t1, targetval*v2, hnode, rules) # ?/v2=targetval
    assert False

hnode='humn'
t1=data['root'][0]
t2=data['root'][2]
if (v1:=check_fixed(t1, hnode, data))!=None:
    res=solve(t2, v1, hnode, data)
else:
    assert (v2:=check_fixed(t2, hnode, data))!=None
    res=solve(t1, v2, hnode, data)
res=int(res)
print(f'part 2: {res}')


In [ ]:
# TEMPLATE
# 2022 day 21
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2022-day-21-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2022-day-21-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
groups=get_line_groups(sample2.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed